In [1]:
import numpy as np
import warnings

import pandas as pd


class Topsis():
    evaluation_matrix = np.array([])  # Matrix
    weighted_normalized = np.array([])  # Weight matrix
    normalized_decision = np.array([])  # Normalisation matrix
    M = 0  # Number of rows
    N = 0  # Number of columns

    '''
	Create an evaluation matrix consisting of m alternatives and n criteria,
	with the intersection of each alternative and criteria given as {\displaystyle x_{ij}}x_{ij},
	we therefore have a matrix {\displaystyle (x_{ij})_{m\times n}}(x_{{ij}})_{{m\times n}}.
	'''

    def __init__(self, evaluation_matrix, weight_matrix, criteria):
        # M×N matrix
        self.evaluation_matrix = np.array(evaluation_matrix, dtype="float")

        # M alternatives (options)
        self.row_size = len(self.evaluation_matrix)

        # N attributes/criteria
        self.column_size = len(self.evaluation_matrix[0])

        # N size weight matrix
        self.weight_matrix = np.array(weight_matrix, dtype="float")
        self.weight_matrix = self.weight_matrix/sum(self.weight_matrix)
        self.criteria = np.array(criteria, dtype="float")

    '''
	# Step 2
	The matrix {\displaystyle (x_{ij})_{m\times n}}(x_{{ij}})_{{m\times n}} is then normalised to form the matrix
	'''

    def step_2(self):
        # normalized scores
        self.normalized_decision = np.copy(self.evaluation_matrix)
        sqrd_sum = np.zeros(self.column_size)
        for i in range(self.row_size):
            for j in range(self.column_size):
                sqrd_sum[j] += self.evaluation_matrix[i, j]**2
        for i in range(self.row_size):
            for j in range(self.column_size):
                self.normalized_decision[i,
                                         j] = self.evaluation_matrix[i, j]/(sqrd_sum[j]**0.5)

    '''
	# Step 3
	Calculate the weighted normalised decision matrix
	'''

    def step_3(self):
        from pdb import set_trace
        self.weighted_normalized = np.copy(self.normalized_decision)
        for i in range(self.row_size):
            for j in range(self.column_size):
                self.weighted_normalized[i, j] *= self.weight_matrix[j]

    '''
	# Step 4
	Determine the worst alternative {\displaystyle (A_{w})}(A_{w}) and the best alternative {\displaystyle (A_{b})}(A_{b}):
	'''

    def step_4(self):
        self.worst_alternatives = np.zeros(self.column_size)
        self.best_alternatives = np.zeros(self.column_size)
        for i in range(self.column_size):
            if self.criteria[i]:
                self.worst_alternatives[i] = min(
                    self.weighted_normalized[:, i])
                self.best_alternatives[i] = max(self.weighted_normalized[:, i])
            else:
                self.worst_alternatives[i] = max(
                    self.weighted_normalized[:, i])
                self.best_alternatives[i] = min(self.weighted_normalized[:, i])

    '''
	# Step 5
	Calculate the L2-distance between the target alternative {\displaystyle i}i and the worst condition {\displaystyle A_{w}}A_{w}
	{\displaystyle d_{iw}={\sqrt {\sum _{j=1}^{n}(t_{ij}-t_{wj})^{2}}},\quad i=1,2,\ldots ,m,}
	and the distance between the alternative {\displaystyle i}i and the best condition {\displaystyle A_{b}}A_b
	{\displaystyle d_{ib}={\sqrt {\sum _{j=1}^{n}(t_{ij}-t_{bj})^{2}}},\quad i=1,2,\ldots ,m}
	where {\displaystyle d_{iw}}d_{{iw}} and {\displaystyle d_{ib}}d_{{ib}} are L2-norm distances 
	from the target alternative {\displaystyle i}i to the worst and best conditions, respectively.
	'''

    def step_5(self):
        self.worst_distance = np.zeros(self.row_size)
        self.best_distance = np.zeros(self.row_size)

        self.worst_distance_mat = np.copy(self.weighted_normalized)
        self.best_distance_mat = np.copy(self.weighted_normalized)

        for i in range(self.row_size):
            for j in range(self.column_size):
                self.worst_distance_mat[i][j] = (self.weighted_normalized[i][j]-self.worst_alternatives[j])**2
                self.best_distance_mat[i][j] = (self.weighted_normalized[i][j]-self.best_alternatives[j])**2
                
                self.worst_distance[i] += self.worst_distance_mat[i][j]
                self.best_distance[i] += self.best_distance_mat[i][j]

        for i in range(self.row_size):
            self.worst_distance[i] = self.worst_distance[i]**0.5
            self.best_distance[i] = self.best_distance[i]**0.5

    '''
	# Step 6
	Calculate the similarity
	'''

    def step_6(self):
        np.seterr(all='ignore')
        self.worst_similarity = np.zeros(self.row_size)
        self.best_similarity = np.zeros(self.row_size)

        for i in range(self.row_size):
            # calculate the similarity to the worst condition
            self.worst_similarity[i] = self.worst_distance[i] / \
                (self.worst_distance[i]+self.best_distance[i])

            # calculate the similarity to the best condition
            self.best_similarity[i] = self.best_distance[i] / \
                (self.worst_distance[i]+self.best_distance[i])
    
    def ranking(self, data):
        return [i+1 for i in data.argsort()]

    def rank_to_worst_similarity(self):
        # return rankdata(self.worst_similarity, method="min").astype(int)
        return self.ranking(self.worst_similarity)

    def rank_to_best_similarity(self):
        # return rankdata(self.best_similarity, method='min').astype(int)
        return self.ranking(self.best_similarity)

    def calc(self):
        print("Step 1\n", self.evaluation_matrix, end="\n\n")
        self.step_2()
        print("Step 2\n", self.normalized_decision, end="\n\n")
        self.step_3()
        print("Step 3\n", self.weighted_normalized, end="\n\n")
        self.step_4()
        print("Step 4\n", self.worst_alternatives,
              self.best_alternatives, end="\n\n")
        self.step_5()
        print("Step 5\n", self.worst_distance, self.best_distance, end="\n\n")
        self.step_6()
        print("Step 6\n", self.worst_similarity,
              self.best_similarity, end="\n\n")

In [2]:
#from topsis import Topsis
import numpy as np

evaluation_matrix = np.array([
])


weights = [5, 5, 9, 0]

'''
if higher value is preferred - True
if lower value is preferred - False
'''
criterias = np.array([True, True, True, True])

t = Topsis(evaluation_matrix, weights, criterias)

t.calc()

print("best_distance\t", t.best_distance)
print("worst_distance\t", t.worst_distance)

# print("weighted_normalized",t.weighted_normalized)

print("worst_similarity\t", t.worst_similarity)
print("rank_to_worst_similarity\t", t.rank_to_worst_similarity())

print("best_similarity\t", t.best_similarity)
print("rank_to_best_similarity\t", t.rank_to_best_similarity())

IndexError: index 0 is out of bounds for axis 0 with size 0

In [3]:
df = pd.read_csv("Data_t.csv")
df.head()

,Unnamed: 0,degree,closeness,betwennes,influence
0,0,0.013599,0.420360,0.000375,0.0
1,1,0.000186,0.295970,0.000000,0.0
2,2,0.413376,0.619361,0.018321,1.0
3,3,0.435358,0.629899,0.020473,1.0
4,4,0.178651,0.506224,0.003627,0.0


In [4]:
df = df.drop('Unnamed: 0', axis=1)

In [5]:
df.head()

,degree,closeness,betwennes,influence
0,0.013599,0.420360,0.000375,0.0
1,0.000186,0.295970,0.000000,0.0
2,0.413376,0.619361,0.018321,1.0
3,0.435358,0.629899,0.020473,1.0
4,0.178651,0.506224,0.003627,0.0


In [6]:
degree = df['degree'].values
closeness = df['closeness'].values
betwennes = df['betwennes'].values
degree

array([1.35991058e-02, 1.86289121e-04, 4.13375559e-01, ...,
       5.21609538e-03, 7.63785395e-03, 5.58867362e-03])

In [12]:
evaluation1_matrix = np.zeros(shape=(len(betwennes), 3))
for d, c, b in zip(degree,closeness,betwennes):
    included= [d,c,b]
    included = np.array(included)
    evaluation1_matrix = np.vstack((evaluation1_matrix, included))


In [8]:
evaluation1_matrix = evaluation1_matrix[1:]

In [13]:
weights = [1, 1, 1]

'''
if higher value is preferred - True
if lower value is preferred - False
'''
criterias = np.array([True, True, True])

In [15]:
t = Topsis(evaluation1_matrix, weights, criterias)

t.calc()

# print("best_distance\t", t.best_distance)
# print("worst_distance\t", t.worst_distance)

# print("weighted_normalized",t.weighted_normalized)

wors = t.rank_to_worst_similarity()

#t.best_similarity)
best = t.rank_to_best_similarity()
print(wors[1])
print(best[0])

Step 1
 [[0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 ...
 [5.21609538e-03 4.15608548e-01 3.72186863e-08]
 [7.63785395e-03 4.16414553e-01 2.63024636e-07]
 [5.58867362e-03 4.15672913e-01 7.35969115e-08]]

Step 2
 [[0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 ...
 [1.70867556e-03 1.50051583e-02 2.78005899e-07]
 [2.50198921e-03 1.50342584e-02 1.96466903e-06]
 [1.83072381e-03 1.50074821e-02 5.49733953e-07]]

Step 3
 [[0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 ...
 [5.69558519e-04 5.00171943e-03 9.26686329e-08]
 [8.33996403e-04 5.01141945e-03 6.54889678e-07]
 [6.10241271e-04 5.00249405e-03 1.83244651e-07]]

Step 4
 [0. 0. 0.] [0.06143095 0.00836816 0.09032255]

Step 5
 [0.         0.         0.   